In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
#viz
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
from sklearn.feature_extraction.text import CountVectorizer

# Any results you write to the current directory are saved as output.

In [2]:
# read file
with open('train.json', 'r') as myfile:
    data=myfile.read()

    # parse file
obj = json.loads(data)
df = pd.DataFrame(obj)

In [3]:
df.head()

,claim,claimant,date,id,label,related_articles
0,A line from George Orwell's novel 1984 predict...,,2017-07-17,0,0,"[122094, 122580, 130685, 134765]"
1,Maine legislature candidate Leslie Gibson insu...,,2018-03-17,1,2,"[106868, 127320, 128060]"
2,A 17-year-old girl named Alyssa Carson is bein...,,2018-07-18,4,1,"[132130, 132132, 149722]"
3,In 1988 author Roald Dahl penned an open lette...,,2019-02-04,5,2,"[123254, 123418, 127464]"
4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]"


In [4]:
#add artical length
df["lens"] = df.claim.str.len()
df.head()

,claim,claimant,date,id,label,related_articles,lens
0,A line from George Orwell's novel 1984 predict...,,2017-07-17,0,0,"[122094, 122580, 130685, 134765]",73
1,Maine legislature candidate Leslie Gibson insu...,,2018-03-17,1,2,"[106868, 127320, 128060]",117
2,A 17-year-old girl named Alyssa Carson is bein...,,2018-07-18,4,1,"[132130, 132132, 149722]",87
3,In 1988 author Roald Dahl penned an open lette...,,2019-02-04,5,2,"[123254, 123418, 127464]",113
4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]",129


In [5]:
#add has author 
df["author"]= np.where(df["claimant"]=="",0,1)
df.head()

,claim,claimant,date,id,label,related_articles,lens,author
0,A line from George Orwell's novel 1984 predict...,,2017-07-17,0,0,"[122094, 122580, 130685, 134765]",73,0
1,Maine legislature candidate Leslie Gibson insu...,,2018-03-17,1,2,"[106868, 127320, 128060]",117,0
2,A 17-year-old girl named Alyssa Carson is bein...,,2018-07-18,4,1,"[132130, 132132, 149722]",87,0
3,In 1988 author Roald Dahl penned an open lette...,,2019-02-04,5,2,"[123254, 123418, 127464]",113,0
4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,2016-03-22,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]",129,1


In [6]:
#replace missing with others first
df["claimant"] = np.where(df["claimant"]=="","other",df["claimant"])

In [7]:
data = df.copy()

In [8]:
#import nltk  <---open up when you first time install nltk
#nltk.download() <---open up when you first time install nltk
import re

In [9]:
def normalize(text):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    text = text.lower()
    # Replace ips
    text = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', text)
    # Isolate punctuation
    text = re.sub(r'([\'\"\.\(\)\!\?\-\\\/\,])', r' \1 ', text)
    # Remove some special characters
    text = re.sub(r'([\;\:\|•«\n])', ' ', text)
    # Remove some special characters
    text = re.sub(r'([\" "\"''"\が\"' '"\、\❂])', ' ', text)
    # Replace numbers and symbols with language
    text = text.replace('&', ' and ')
    text = text.replace('@', ' at ')
    text = text.replace('-', '')
    text = text.replace('“', '')
    text = text.replace("'", '')  
       
    #text = text.replace('0', ' zero ')
    #text = text.replace('1', ' one ')
    #text = text.replace('2', ' two ')
    #text = text.replace('3', ' three ')
    #text = text.replace('4', ' four ')
    #text = text.replace('5', ' five ')
    #text = text.replace('6', ' six ')
    #text = text.replace('7', ' seven ')
    #text = text.replace('8', ' eight ')
    #text = text.replace('9', ' nine ')
    return text
data['claim'] = data['claim'].apply(normalize)

In [10]:
#function to remove punctuation
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

data['claim'] = data['claim'].apply(remove_punctuation)

In [11]:
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')
nlp.Defaults.stop_words |= {"s","t","el","…","f'","u'","1'","ve","u"}

In [12]:
# Clean text before feeding it to spaCy
punctuations = string.punctuation

# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in STOP_WORDS and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [13]:
# Clean up all text
data["claim"] = cleanup_text(data.claim, logging = True)

Processed 1000 out of 15555 documents.
Processed 2000 out of 15555 documents.
Processed 3000 out of 15555 documents.
Processed 4000 out of 15555 documents.
Processed 5000 out of 15555 documents.
Processed 6000 out of 15555 documents.
Processed 7000 out of 15555 documents.
Processed 8000 out of 15555 documents.
Processed 9000 out of 15555 documents.
Processed 10000 out of 15555 documents.
Processed 11000 out of 15555 documents.
Processed 12000 out of 15555 documents.
Processed 13000 out of 15555 documents.
Processed 14000 out of 15555 documents.
Processed 15000 out of 15555 documents.


In [14]:
# claim# claim
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=6, norm='l2', encoding='latin-1', ngram_range=(1, 2),max_features=None, stop_words='english')
features = tfidf.fit_transform(data.claim).toarray()
features.shape

(15555, 5864)

In [15]:
from joblib import dump
dump(tfidf,'pre_process_nov4.joblib')


['pre_process_nov4.joblib']

In [16]:
from sklearn.model_selection import train_test_split, KFold
labels = data.label
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)

In [17]:
from imblearn.over_sampling import SMOTE
#Resampling:
smote = SMOTE('minority')
# sm = SMOTE({0:4941, 1:4941, 2:4941}, random_state=123)
X_sm, y_sm = smote.fit_sample(X_train, y_train)
print(X_sm.shape, y_sm.shape)

Using TensorFlow backend.


(14233, 5864) (14233,)


In [18]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
model_rs = LinearSVC()
model_rs.fit(X_sm,y_sm)
y_pred_rs = model_rs.predict(X_test)
print(classification_report(y_test,y_pred_rs))

              precision    recall  f1-score   support

           0       0.62      0.59      0.60      2467
           1       0.55      0.50      0.53      2100
           2       0.18      0.28      0.22       567

    accuracy                           0.52      5134
   macro avg       0.45      0.46      0.45      5134
weighted avg       0.55      0.52      0.53      5134



In [19]:
# save the preprocess to disk
dump(model_rs,'model_nov4.joblib')

['model_nov4.joblib']